In [4]:
import numpy as np
import pandas as pd
from definitions import *
from tqdm import tqdm


In [17]:
points_above_average = pd.read_csv("./points_above_average.csv", index_col=0)

In [18]:
users_agregated_data = pd.DataFrame()

In [19]:
users = os.listdir(PATTERN_TEST_PATH_SAME_POINT)
data = [ ["user", users]]
for user in tqdm(users):
    arqs = os.listdir(PATTERN_TEST_PATH_SAME_POINT + "/" + user)
    for arq in arqs:
        mask_ref = arq.split("_")[0]
        if(mask_ref == "00-00"):
            continue
        user_mask = pd.read_csv(PATTERN_TEST_PATH_SAME_POINT + "/" + user+"/"+arq)
        pav_list = points_above_average["points_above_average"].tolist()
        for pav in pav_list:
            achou = False
            for i in range(len(data)):
                if(data[i][0] == mask_ref + "_" + str(pav)):
                    achou = True
            if(not achou):
                data.append([mask_ref + "_" + str(pav),[]])
            
            for i in range(len(data)):
                if(data[i][0] == mask_ref + "_" + str(pav)):
                    data[i][1].append(user_mask["distances"].tolist()[int(pav)])
                
for i in range(len(data)):
    print(len(data[i][1]))
    users_agregated_data[data[i][0]] = np.array(data[i][1])


100%|██████████| 309/309 [10:29<00:00,  2.04s/it]
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 

309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309


C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

309
309
309
309
309
309
309
309
309
309
309
309
309
309


C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

...

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

309
309
309
309
309
309
309
309
309
309
309
309

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which


309
309
309
309
309
309
309


C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2436\2297437362.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  users_agregated_data[data[i][0]] = np.array(data[i][1])


In [16]:
# Mano o df ta com 1233 colunas... isso aqui n ta legal não kkkkk
print(len(data))
for i in range(len(data)):
    print(len(data[i][1]))

1233
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309
309

In [10]:
users_agregated_data.to_csv("./aregated_data.csv")

In [14]:
users_agregated_data.to_excel("./agregated_data.xlsx", engine='xlsxwriter')  